# 04 – Kombin Öneri Sistemi Demosu

Bu defterde eğitilmiş hibrit modeli kullanarak seçilen bir ürün için en uyumlu kombinleri önereceğiz.

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
from torchvision import transforms

from src.models.hybrid_model import HybridColorModel

df_colors = pd.read_csv('data/processed/colors.csv')
df_colors.head()

In [ ]:
# Modeli yükle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridColorModel(color_dim=15).to(device)
state_path = 'results/hybrid/best_model.pt'
if os.path.exists(state_path):
    model.load_state_dict(torch.load(state_path, map_location=device))
    print('Model yüklendi:', state_path)
else:
    print('UYARI: Model ağırlığı bulunamadı, önce 03_hybrid_model_training.ipynb çalıştırılmalı.')

model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [ ]:
# Embedding çıkaran yardımcı fonksiyon
def get_embedding_for_row(row):
    img = Image.open(row['image_path']).convert('RGB')
    img_t = transform(img).unsqueeze(0).to(device)

    cols = [f'c{i}' for i in range(15)]
    color_vec = torch.tensor(row[cols].values.astype('float32')).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_t, color_vec)
        score = outputs['score'].item()
    return score

In [ ]:
# Basit öneri fonksiyonu (örnek)
def recommend_combinations(query_idx, top_k=5):
    query_row = df_colors.iloc[query_idx]
    scores = []

    for idx, row in df_colors.iterrows():
        if idx == query_idx:
            continue
        s = get_embedding_for_row(row)
        scores.append((idx, s))

    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return scores[:top_k]

recommendations = recommend_combinations(0, top_k=5)
recommendations

In [ ]:
# Örnek ürün ve önerilen kombinlerin görselleştirilmesi (örnek)
def show_recommendations(query_idx, recs):
    n = len(recs) + 1
    plt.figure(figsize=(3*n, 4))

    # Sorgu ürünü
    query_img = Image.open(df_colors.iloc[query_idx]['image_path']).convert('RGB')
    plt.subplot(1, n, 1)
    plt.imshow(query_img)
    plt.axis('off')
    plt.title('Sorgu')

    # Öneriler
    for i, (idx, score) in enumerate(recs, start=2):
        img = Image.open(df_colors.iloc[idx]['image_path']).convert('RGB')
        plt.subplot(1, n, i)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f'score={score:.2f}')

    plt.tight_layout()
    os.makedirs('results/figures', exist_ok=True)
    plt.savefig('results/figures/recommendation_example.png')
    print('Kaydedildi: results/figures/recommendation_example.png')

show_recommendations(0, recommendations)